# Compare with R nptest

In [1]:
import sys
import pandas as pd
import numpy as np
from scipy import stats, io
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg

In [2]:
sys.path.append('../../scripts')

In [3]:
import fl_permute as fl
import utils

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_style('ticks')
sns.set_context('notebook')
sns.set_palette("colorblind")

Load group average connectivity and dynamics metrics (and ROI volumes).

In [5]:
roi_vols = np.genfromtxt('../../../data/resources/BNA_ROI_volumes.txt')

In [6]:
sc_degree_df = pd.read_csv('../../../results/Yeh_BNA_EIR_qa_nothresh_centralities.tsv', delimiter='\t', index_col=0)

In [7]:
fc_degree_df = pd.read_csv('../../../results/NKI-1400_CompCorGSR_BNA_centralities.tsv', delimiter='\t', index_col=0)

In [8]:
ts_df = pd.read_csv('../../../results/NKI-1400_CompCorGSR_BNA_timescales_median.csv')

In [9]:
fc_pc = np.genfromtxt('../../../results/NKI_1400_CompCorGSR_BNA_zmat_mean_po_Louvain/MegaPart_gamma2p2_PC.txt')

In [10]:
fc_wmd = np.genfromtxt('../../../results/NKI_1400_CompCorGSR_BNA_zmat_mean_po_Louvain/MegaPart_gamma2p2_WMDz.txt')

In [11]:
sc_pc = np.genfromtxt('../../../results/Yeh_BNA_EIR_qa_nothresh_Louvain/gamma1p2_PC.txt')

In [12]:
sc_wmd = np.genfromtxt('../../../results/Yeh_BNA_EIR_qa_nothresh_Louvain/gamma1p2_WMDz.txt')

In [13]:
dat = pd.DataFrame(np.array([sc_degree_df.strength.values, fc_degree_df.strength_zmean_po.values,
                             ts_df.decay10_median.values, ts_df.decay20_median.values, ts_df.lag1_median.values, 
                             sc_wmd, fc_wmd, fc_pc, sc_pc, roi_vols]).T,
                   columns = ['sc_strength', "fc_strength", 'decay10', 'decay20', 'lag1', 'sc_wmd',
                              'fc_wmd', 'fc_pc', 'sc_pc', 'roi_vols'])

### Single X, Single Z

In [18]:
x1z1_fval, x1z1_pvals, x1z1_model, x1z1_fnull = fl.mlr_perm(dat, 'fc_strength', ['sc_wmd'], ['roi_vols'], stat='fstat',
                                                            n_perms=100000, return_null=True, return_surrogates=False)

In [19]:
x1z1_fval

4.78683633625097

In [20]:
x1z1_pvals

p_greater    0.02984
dtype: float64

### Single X, Multiple Z

In [22]:
x1z2_fval, x1z2_pvals, x1z2_model, x1z2_fnull = fl.mlr_perm(dat, 'fc_strength', ['sc_wmd'], ['roi_vols', 'fc_pc'], stat='fstat',
                                                            n_perms=100000, return_null=True, return_surrogates=False)

In [23]:
x1z2_fval

1.183740087106998

In [24]:
x1z2_pvals

p_greater    0.276257
dtype: float64

### Multiple X, Multiple Z

In [25]:
x2z2_fval, x2z2_pvals, x2z2_model, x2z2_fnull = fl.mlr_perm(dat, 'fc_strength', ['sc_wmd', 'sc_pc'], ['roi_vols','fc_pc'], stat='fstat',
                                                            n_perms=100000, return_null=True, return_surrogates=False)

In [26]:
x2z2_fval

1.6504957522078691

In [27]:
x2z2_pvals

p_greater    0.194618
dtype: float64

### Multiple X, Single Z

In [28]:
x2z1_fval, x2z1_pvals, x2z1_model, x2z1_fnull = fl.mlr_perm(dat, 'fc_strength', ['sc_wmd', 'sc_pc'], ['roi_vols'], stat='fstat',
                                                            n_perms=100000, return_null=True, return_surrogates=False)

In [29]:
x2z1_fval

3.6331784357883343

In [31]:
x2z1_pvals

p_greater    0.02831
dtype: float64